In [1]:
import numpy as np, pandas as pd, json, itertools, ast, sys, csv, random
from itertools import permutations
from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat
from functions.RL_functions import RL_Actions, RL_Initial_Step, RL_available_actions, \
                                    RL_Next_step, RL_Reward, RL_Rebuild_Dictionary, \
                                    RL_Argmax, RL_First_Policy


URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_5.4_full'

In [2]:
df_raw = pd.read_csv(URL + csv_name + ".csv", engine='python', encoding='utf-8')
if 'sparql_file' in df_raw.columns:
    df_raw = df_raw.rename(columns={"sparql_file": "query"})

print(df_raw.columns)


Index(['unique_id', 'filename', 'query', 'profile', 'limit', 'group_by',
       'distinct', 'order_by', 'union', 'left_join', 'join', 'iter', 'filter',
       'num_filter', 'filter_eq', 'filter_gt', 'filter_ge', 'filter_lt',
       'filter_le', 'filter_neq', 'filter_iri', 'filter_neq.1', 'filter_bound',
       'filter_contains', 'filter_exists', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_not',
       'filter_notexists', 'filter_regex', 'filter_sameTerm', 'filter_str',
       'filter_strstarts', 'filter_or', 'filter_and', 'time', 'cpu_p', 'rnd',
       'seq', 'same_seg_p', 'same_page_p', 'disk_reads', 'read_ahead', 'wait',
       'comp_msec', 'comp_reads', 'comp_read_p', 'comp_messages', 'comp_clw',
       'triples', 'total_bgps', 'treesize', 'matrix_format', 'trees',
       'trees_old_format', 'json_time_predicate', 'json_fanout_predicate',
       'json_input_rows_predicate', 'json_cardinality_fanout',
       'json_cardinal

In [3]:
example_idx = []
for i in range(len(df_raw)):
    if df_raw['total_bgps'][i] > 2 and df_raw['triples'][i] > 3:
        example_idx.append(i)
kkk = random.choice(example_idx)

In [4]:
print(kkk)

12176


In [5]:
df_raw['trees'][kkk]

'["LEFT_JOIN", ["LEFT_JOIN", ["LEFT_JOIN", ["URI_URI_VARᶲIRI_ID;...direct/P585;"], ["URI_URI_VARᶲIRI_ID;...direct/P625;"]], ["URI_URI_VARᶲIRI_ID;...direct/P131;"]], ["URI_URI_VARᶲIRI_ID;...direct/P31;"]]'

In [6]:
bgp = ast.literal_eval(df_raw['bgps'][kkk])

In [7]:
print(bgp)

{'bgp_1': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P585"', 'O': 'None', 'time': 10.0, 'fanout': 1.0, 'input_rows': 1.0, 'cardinality_estimate': 0, 'cardinality_fanout': 0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 1, 'OP': 'OP6'}], 'opt': 1}, 'bgp_2': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P625"', 'O': 'None', 'time': 3.8, 'fanout': 0.0, 'input_rows': 1.0, 'cardinality_estimate': 1.0, 'cardinality_fanout': 0.0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 2, 'OP': 'OP9'}], 'opt': 1}, 'bgp_3': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P131"', 'O': 'None', 'time': 5.6, 'fanout': 0.0, 'input_rows': 1.0, 'cardinality_estimate': 1.0, 'cardinality_fanout': 0.0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 3, 'OP': 'OP12'}], 'opt': 1}, 'bgp_4': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P31"', 'O': 'None', 'time': 8.5, 'fanout': 2.0, 'input_rows

In [8]:
actions = RL_Actions(bgp)
print("Actions")
print(actions)

Actions
[(0, 'bgp_1', 'IRI_ID"...direct/P585"', 'URI_URI_VAR'), (1, 'bgp_2', 'IRI_ID"...direct/P625"', 'URI_URI_VAR'), (2, 'bgp_3', 'IRI_ID"...direct/P131"', 'URI_URI_VAR'), (3, 'bgp_4', 'IRI_ID"...direct/P31"', 'URI_URI_VAR')]


In [49]:
actions = RL_Actions(bgp)
current_state = []
reward = 0
terminal = False
while not terminal:
    current_state, reward, terminal = RL_Next_step(actions,current_state,reward,-1)
    
final_state = current_state.copy()
print("final current_state", current_state)
print("final reward",reward)
print("actions",len(actions),"current_state",len(current_state))
final_state_predicates = [i[2] for i in final_state]
print("final_state_predicates", final_state_predicates)
new_dicto = RL_Rebuild_Dictionary(bgp, final_state)
print(new_dicto)
print(len(new_dicto.keys()))

new_tree = str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')
new_tree

print("nuevo arbol", new_tree)

final current_state [(3, 'bgp_4', 'IRI_ID"...direct/P31"', 'URI_URI_VAR'), (0, 'bgp_1', 'IRI_ID"...direct/P585"', 'URI_URI_VAR'), (1, 'bgp_2', 'IRI_ID"...direct/P625"', 'URI_URI_VAR'), (2, 'bgp_3', 'IRI_ID"...direct/P131"', 'URI_URI_VAR')]
final reward -5
actions 4 current_state 4
final_state_predicates ['IRI_ID"...direct/P31"', 'IRI_ID"...direct/P585"', 'IRI_ID"...direct/P625"', 'IRI_ID"...direct/P131"']
{'bgp_4': {'bgp_list': [{'P': 'IRI_ID"...direct/P31"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_2': {'bgp_list': [{'P': 'IRI_ID"...direct/P625"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_3': {'bgp_list': [{'P': 'IRI_ID"...direct/P131"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_1': {'bgp_list': [{'P': 'IRI_ID"...direct/P585"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}}
4
nuevo arbol ["LEFT_JOIN", ["LEFT_JOIN", ["LEFT_JOIN", ["URI_URI_VARᶲIRI_ID;...direct/P31;"], ["URI_URI_VARᶲIRI_ID;...direct/P625;"]], ["URI_URI_VARᶲIRI_ID;...direct/P131;"]], ["URI_URI_VARᶲIRI_ID;.

{'bgp_2': {'bgp_list': [{'P': 'IRI_ID"...direct/P625"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_4': {'bgp_list': [{'P': 'IRI_ID"...direct/P31"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_3': {'bgp_list': [{'P': 'IRI_ID"...direct/P131"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}, 'bgp_1': {'bgp_list': [{'P': 'IRI_ID"...direct/P585"', 'triple_type': 'URI_URI_VAR'}], 'opt': 1}}
4


In [12]:
print(bgp)

{'bgp_1': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P585"', 'O': 'None', 'time': 10.0, 'fanout': 1.0, 'input_rows': 1.0, 'cardinality_estimate': 0, 'cardinality_fanout': 0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 1, 'OP': 'OP6'}], 'opt': 1}, 'bgp_2': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P625"', 'O': 'None', 'time': 3.8, 'fanout': 0.0, 'input_rows': 1.0, 'cardinality_estimate': 1.0, 'cardinality_fanout': 0.0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 2, 'OP': 'OP9'}], 'opt': 1}, 'bgp_3': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P131"', 'O': 'None', 'time': 5.6, 'fanout': 0.0, 'input_rows': 1.0, 'cardinality_estimate': 1.0, 'cardinality_fanout': 0.0, 'triple_type': 'URI_URI_VAR', 'optional_section?': 1, 'num_bgp': 3, 'OP': 'OP12'}], 'opt': 1}, 'bgp_4': {'bgp_list': [{'S': 'IRI_ID"...Q6902"', 'P': 'IRI_ID"...direct/P31"', 'O': 'None', 'time': 8.5, 'fanout': 2.0, 'input_rows

In [13]:
symbol = 'ᶲ'


In [14]:
new_dicto

{'bgp_2': {'bgp_list': [{'P': 'IRI_ID"...direct/P625"',
    'triple_type': 'URI_URI_VAR'}],
  'opt': 1},
 'bgp_4': {'bgp_list': [{'P': 'IRI_ID"...direct/P31"',
    'triple_type': 'URI_URI_VAR'}],
  'opt': 1},
 'bgp_3': {'bgp_list': [{'P': 'IRI_ID"...direct/P131"',
    'triple_type': 'URI_URI_VAR'}],
  'opt': 1},
 'bgp_1': {'bgp_list': [{'P': 'IRI_ID"...direct/P585"',
    'triple_type': 'URI_URI_VAR'}],
  'opt': 1}}

In [15]:
test_tree_format = str(TreeFormat(bgp,symbol)).replace('"', ';').replace("'", '"')
test_tree_format

'["LEFT_JOIN", ["LEFT_JOIN", ["LEFT_JOIN", ["URI_URI_VARᶲIRI_ID;...direct/P585;"], ["URI_URI_VARᶲIRI_ID;...direct/P625;"]], ["URI_URI_VARᶲIRI_ID;...direct/P131;"]], ["URI_URI_VARᶲIRI_ID;...direct/P31;"]]'

In [16]:
old_tree = df_raw['trees'][kkk]
old_tree

'["LEFT_JOIN", ["LEFT_JOIN", ["LEFT_JOIN", ["URI_URI_VARᶲIRI_ID;...direct/P585;"], ["URI_URI_VARᶲIRI_ID;...direct/P625;"]], ["URI_URI_VARᶲIRI_ID;...direct/P131;"]], ["URI_URI_VARᶲIRI_ID;...direct/P31;"]]'

In [17]:
old_tree == test_tree_format

True

In [18]:
new_tree = str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')
new_tree

'["LEFT_JOIN", ["LEFT_JOIN", ["LEFT_JOIN", ["URI_URI_VARᶲIRI_ID;...direct/P625;"], ["URI_URI_VARᶲIRI_ID;...direct/P31;"]], ["URI_URI_VARᶲIRI_ID;...direct/P131;"]], ["URI_URI_VARᶲIRI_ID;...direct/P585;"]]'

In [19]:
new_tree == old_tree

False

In [20]:
listy = [7, 6, 5, 7, 6, 7, 6, 6, 6, 4, 5, 6,55]

12

In [23]:
### q(state,action), en mi caso, son del mismo tamaño
### cada fila es un estado, cada columna es un accion
q_values = RL_First_Policy(actions)

In [35]:
q_values[0]

array([0., 0., 0., 0.])